# Metrical Analysis of Sanskrit Ninth Class Verb Forms

Our main stages:

1. Raw Corpus
2. Roots
3. Roots with stanza attestations
4. Verse lines (and enrichment)
5. Verse lines with metrical analysis
6. Analysis

with validation in each stage...